# SCC

stochastic correlation check

## Import

In [1]:
import random, struct, math
import torch
import numpy as np
import torch.nn as nn 
import argparse
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Parser

In [2]:
parser = argparse.ArgumentParser(description='fixed_mac')
parser.add_argument('--full_bits', type=int, default=8, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=4, help='Number of Quantization Bits')
parser.add_argument('--bBW', type=int, default=4, help='Number of bit width')
args = parser.parse_args(args=[])

## Definition

### int2bin : int number to binary number

In [3]:
def int2bin(iIn,iBW):
    iBW = iBW + 1
    if iIn >= 0:
        bOut = bin(iIn).replace('0b','').rjust(iBW,'0')
    else :
        bOut = bin(iIn & (pow(2,iBW)-1)).replace('0b','').rjust(iBW,'1')
    return bOut[1:]

### XOR : xor gate with string

In [4]:
def XOR(iA,iB):
    if iA != iB :
        iOut = '1'
    else : 
        iOut = '0'
    return iOut

### SNUM : sign number determination

In [5]:
def snum(a):
    if a >= 0 :
        return '0'
    else :
        return '1'

### fxp & flp2fix : floating number to fixed number conversion

In [6]:
class fxp:
    def __init__(self, bIn, iBWF):
        self.iFullBW = len(bIn)
        self.iIntgBW = self.iFullBW - iBWF
        self.bSign = bIn[0]
        self.bIntg = bIn[:self.iIntgBW]
        self.bFrac = bIn[self.iIntgBW:]
        self.fFull = 0
        try:
            for idx, bit in enumerate(bIn):
                if idx == 0:
                    self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
                else:
                    self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
        except:
            print(bIn)
        self.dispFull = self.bIntg +"."+ self.bFrac 
        return

In [7]:
class flp2fix:
    def __init__(self, fIn, iBW, iBWF):
        self.fMin = - 2 ** (iBW - iBWF - 1)
        self.fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
        self.fResol = 2 ** -iBWF
        if fIn < self.fMin or fIn > self.fMax:
            print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
        self.iBW = iBW
        self.iBWI = iBW - iBWF
        self.iBWF = iBWF

        self.iFLP2INT = abs(int(fIn * 2 ** iBWF))
        if fIn < 0:
            self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

        if fIn >= 0:
            self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
        else:
            self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
            if len(self.bFull) > iBW:
                self.bFull = '0' * iBW

        self.cssFxp = fxp(self.bFull, self.iBWF)
        self.bSign = self.cssFxp.bSign
        self.bIntg = self.cssFxp.bIntg
        self.bFrac = self.cssFxp.bFrac
        self.fFull = self.cssFxp.fFull
        return

In [8]:
def flp2fixTensor(fIn, iBW, iBWF):
    fMin = - 2 ** (iBW - iBWF - 1)
    fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
    fList = []
    for aTensor in fIn.view(-1):
        fList.append(flp2fix(aTensor, iBW, iBWF).fFull)
    return torch.tensor(fList).view(fIn.size())

### LFSR : make pseudorandom number bitstream

In [9]:
class LFSR:
    def Random(self):
        self.b0 = eval(f'str(random.randint(0,1))')
        self.b1 = eval(f'str(random.randint(0,1))')
        self.b2 = eval(f'str(random.randint(0,1))')
        self.b3 = eval(f'str(random.randint(0,1))')
        
        return self.b0 + self.b1 + self.b2 + self.b3 
    
    def Normal(self,stream):
        self.b0 = XOR(int(stream[2]),int(stream[3]))
        self.b1 = stream[0]
        self.b2 = stream[1]
        self.b3 = stream[2]
        
        return self.b0 + self.b1 + self.b2 + self.b3 
    
    def Allzero(self):
        self.b0 = '0'
        self.b1 = '0'
        self.b2 = '0'
        self.b3 = '0'
        
        return self.b0 + self.b1 + self.b2 + self.b3 

### LFSRlist : make pseudorandom number bitstream with 2**bBW cycle

In [10]:
def LFSRlist():
    lfsr = LFSR()
    lfsrlist = []
    for k in range(2**(args.bBW)-1): #lfsr number generating
        if k == 0:
            lfsrlist.append(lfsr.Random())
        else :
            lfsrlist.append(lfsr.Normal(lfsrlist[k-1]))
        if (k == 2**(args.bBW)-2):
            lfsrlist.append(lfsr.Allzero())
    
    if (args.bBW) != args.frac_bits :
        if args.bBW < args.frac_bits :
            for i in range(len(lfsrlist)):
                lfsrlist[i] = lfsrlist[i] + (args.frac_bits-args.bBW)*'0'
        else :
            print("it can't work")
            return 0
    
    return lfsrlist

### Comp : Comparator in SNG

In [11]:
def Comp(a,lfsr,snum):
    for com in range(0,len(a)):
        oA = '0'
        if a[com]!=lfsr[com]:
            if(int(a[com]) > int(lfsr[com])):
                oA = '1'
            break
    return XOR(oA,snum)

### Perm : module for permutation SNG 

In [12]:
def perm(a):
    al = len(a)
    blist = []
    for i in range(al) :
        #print(al-i-1)
        blist.append(a[al-i-1])
    
    b = "".join(blist)
    
    return b

### SNG : stochastic number generator module

In [13]:
def SNG(iIN,lfsr):

    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        lNUM = lfsr[k]
        a = Comp(bIN[-(args.frac_bits):],lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    if sNUM == '1' and bIN == args.full_bits*'0' :
        return '1'+(2**(args.bBW))*'0'
    else :
        return sA

### SNG_P : permutation stochastic number generator module

In [14]:
def SNG_P(iIN,lfsr):
    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        if (args.bBW == args.frac_bits) :    
            lNUM = perm(lfsr[k])
        elif (args.bBW < args.frac_bits) :
            lNUM = perm(lfsr[k][:args.bBW])+(args.frac_bits-args.bBW)*"0"
        a = Comp(bIN[-(args.frac_bits):],lNUM,sNUM)
        oAlist.append(Comp(bIN[-(args.frac_bits):],lNUM,sNUM)) #comparator of input a
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    if sNUM == '1' and bIN == args.full_bits*'0' :
        return '1'+(2**(args.bBW))*'0'
    else :
        return sA

### SNGnumpy : SNG module with numpy

In [15]:
def SNGnumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG(float(aNumpy),lfsr))
                     
    return np.array(sList).reshape(fIn.size())

### SNGpnumpy : permutation SNG module with numpy

In [16]:
def SNGpnumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG_P(float(aNumpy),lfsr))
                     
    return  np.array(sList).reshape(fIn.size())

In [17]:
def CountOne1(nIn):
    n = 0
    for num in nIn:
        if num == '1' :
            n += 1
    return n

In [18]:
def Percent(nIn):
    i = CountOne1(nIn)
    leng = len(nIn)
    return i/leng

In [19]:
def ANDstr(a,b):
    al = len(a)
    bl = len(b)
    
    outlist = []
    
    if al != bl :
        print("length of string is different")
        return 0

    for i in range(al-1) :
        outlist.append(str(int(a[i+1]) & int(b[i+1])))
    
    #print(outlist)
    out = "".join(outlist)
    
    return out    

## SCC

### Delta

$ \delta(S{x},S{y})= P(S{x}\wedge S{y}) - P(S{x})P(S{y}) $

In [20]:
def delta(a,b):
    Pand = Percent(ANDstr(a,b))
    Por = Percent(a[1:])*Percent(b[1:])
    return Pand - Por

### SCC

$ \left\{\begin{matrix}\frac{\delta (S{x},S{y})}{min(P(S{x}),P(S{y}))-P(S{x})P(S{y})}
 \\0
 \\\frac{\delta (S{x},S{y})}{P(S{x})P(S{y})-max(P(S{x})+P(S{y})-1,0)}
\end{matrix}\right. $

In [21]:
def SCC(a,b):
    lfsr = LFSRlist()
    
    sA = SNG(a,lfsr)
    sB = SNG(b,lfsr)
    
    PerA = Percent(sA[1:])
    PerB = Percent(sB[1:])
    delt = delta(sA,sB)
    if delt > 0 :
        return delt / (min(PerA,PerB)-(PerA*PerB))
    elif delt < 0 :
        return delt / ((PerA*PerB)-max(PerA+PerB-1,0))
    else :
        return 0

In [22]:
def SCCp(a,b):
    lfsr = LFSRlist()
    
    sA = SNG(a,lfsr)
    sB = SNG_P(b,lfsr)
    
    PerA = Percent(sA[1:])
    PerB = Percent(sB[1:])
    delt = delta(sA,sB)
    if delt > 0 :
        return delt / (min(PerA,PerB)-(PerA*PerB))
    elif delt < 0 :
        return delt / ((PerA*PerB)-max(PerA+PerB-1,0))
    else :
        return 0

In [23]:
def SCCpTensor(A,B):
    SCClist = []
    for aTensor,bTensor in zip(A.view(-1),B.view(-1)):
        SCClist.append(SCCp(aTensor,bTensor))
    return torch.tensor(SCClist).view(A.size())

In [24]:
def SCCTensor(A,B):
    SCClist = []
    for aTensor,bTensor in zip(A.view(-1),B.view(-1)):
        SCClist.append(SCC(aTensor,bTensor))
    return torch.tensor(SCClist).view(A.size())

## SCC Tensor TEST

In [25]:
A = torch.rand((10,15))
B = torch.rand((10,15))

In [26]:
C = SCCpTensor(A,B) # C = permutation LFSR

In [27]:
D = SCCTensor(A,B) # D = original each LFSR

In [35]:
for a,b in zip(C.view(-1),D.view(-1)):
    print(f'-----------------------------------------')
    print(f'SCC of permutation LFSR is : {a}')
    print(f'SCC of original LFSR is    : {b}')

print(f'SCC of permutation LFSR mean is : {torch.mean(C)}')
print(f'SCC of original LFSR mean is    : {torch.mean(D)}')

-----------------------------------------
SCC of permutation LFSR is : 0.20000000298023224
SCC of original LFSR is    : 1.0
-----------------------------------------
SCC of permutation LFSR is : 0.40740740299224854
SCC of original LFSR is    : 1.0
-----------------------------------------
SCC of permutation LFSR is : 1.0
SCC of original LFSR is    : 1.0
-----------------------------------------
SCC of permutation LFSR is : 1.0
SCC of original LFSR is    : 1.0
-----------------------------------------
SCC of permutation LFSR is : 0.0
SCC of original LFSR is    : 0.0
-----------------------------------------
SCC of permutation LFSR is : 0.0
SCC of original LFSR is    : 0.0
-----------------------------------------
SCC of permutation LFSR is : 0.1111111119389534
SCC of original LFSR is    : 1.0
-----------------------------------------
SCC of permutation LFSR is : 1.0
SCC of original LFSR is    : 1.0
-----------------------------------------
SCC of permutation LFSR is : 1.0
SCC of origina